# Pretty Printing Helper

In [232]:
import json

def show_json(message, obj):
    display(message, json.loads(obj.model_dump_json()))
    

# Creating Client

In [233]:
from openai import OpenAI
from dotenv import find_dotenv, load_dotenv
from datetime import datetime, timedelta

_:bool = load_dotenv(find_dotenv())
client:OpenAI = OpenAI()

# Function calling

In [234]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def getCurrentWeather(location:str, unit:str="fahrenheit")->str | dict | None:
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "los angeles" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})
    

def getNickname(location:str)->str:
    """Get the nickname of a city"""
    if "tokyo" in location.lower():
        return "tk"
    elif "los angeles" in location.lower():
        return "la"
    elif "paris" in location.lower():
        return "py"
    else:
        return location

def get_flight_info(loc_origin, loc_destination):
    """Get flight information between two locations."""

    # Example output returned from an API or database
    flight_info = {
        "loc_origin": loc_origin,
        "loc_destination": loc_destination,
        "datetime": str(datetime.now() + timedelta(hours=2)),
        "airline": "AirBlue",
        "flight": "AB643",
    }

    return json.dumps(flight_info) 

# Step 1: Create an Assistant and register/report your functions

In [235]:
from openai.types.beta import Assistant

assistant: Assistant = client.beta.assistants.create(
  name="Weather Assistant",  
  instructions="You are a weather bot. Use the provided functions to answer questions.",
  model="gpt-3.5-turbo-1106",
  tools=[{
      "type": "function",
    "function": {
      "name": "getCurrentWeather",
      "description": "Get the weather in location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
          "unit": {"type": "string", "enum": ["c", "f"]}
        },
        "required": ["location"]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "getNickname",
      "description": "Get the nickname of a city",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
        },
        "required": ["location"]
      }
    } 
  },
   {
        "type": "function",
        "function": {
            "name": "get_flight_info",
        "description": "Get flight information between two locations",
        "parameters": {
            "type": "object",
            "properties": {
                "loc_origin": {
                    "type": "string",
                    "description": "The departure airport, e.g. DUS",
                },
                "loc_destination": {
                    "type": "string",
                    "description": "The destination airport, e.g. HAM",
                },
            },
            "required": ["loc_origin", "loc_destination"],
        },
        }
    }
  ]
)

# Step 2: Create a Thread

In [236]:
from openai.types.beta.thread import Thread

thread: Thread  = client.beta.threads.create()

dict(thread)

{'id': 'thread_C41ap9vCkJ5hiPmhJuyHckJ7',
 'created_at': 1702036316,
 'metadata': {},
 'object': 'thread'}

# Step 3: Add a Message to a Thread

In [250]:
from openai.types.beta.threads.thread_message import ThreadMessage

# First Request
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What is the nickname and weather of Tokyo and when is next flight from Lahore to Karachi?",
)
dict(message)

{'id': 'msg_E3NLrGIHNEDIl1hR7lZaW87C',
 'assistant_id': None,
 'content': [MessageContentText(text=Text(annotations=[], value='What is the nickname and weather of Tokyo and when is next flight from Lahore to Karachi?'), type='text')],
 'created_at': 1702036999,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_C41ap9vCkJ5hiPmhJuyHckJ7'}

# Step 4: Run the Assistant

In [251]:
from openai.types.beta.threads.run import Run

run: Run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)
dict(run)

{'id': 'run_WgM9KxUmMkst5FkHLc6lQjqX',
 'assistant_id': 'asst_e1sGZ1kqmKT6lMEyvnw8Sn3C',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1702037001,
 'expires_at': 1702037601,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a weather bot. Use the provided functions to answer questions.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_C41ap9vCkJ5hiPmhJuyHckJ7',
 'tools': [ToolAssistantToolsFunction(function=FunctionDefinition(name='getCurrentWeather', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state e.g. San Francisco, CA'}, 'unit': {'type': 'string', 'enum': ['c', 'f']}}, 'required': ['location']}, description='Get the weather in location'), type='function'),
  ToolAssistantToolsFunction(function=FunctionDefinition(name='getNickname', parameters={'type': 'object

# Avaliable function

In [252]:
available_functions = {
    "getCurrentWeather": getCurrentWeather,
    "getNickname": getNickname,
    "get_flight_info": get_flight_info
} 

# Step 5: Polling for Updates and Calling Functions

In [253]:
import time

  # Loop until the run completes or requires action
while True:
    runStatus = client.beta.threads.runs.retrieve(thread_id=thread.id,
                                                  run_id=run.id)
    # Add run steps retrieval here for debuging
    run_steps = client.beta.threads.runs.steps.list(thread_id=thread.id, run_id=run.id)
    # show_json("Run Steps:", run_steps)
    print(runStatus.status ,'.....')

    # This means run is making a function call   
    if runStatus.status == "requires_action":
        print(runStatus.status ,'.....')
        print("Status: ", "requires_action")
        show_json("submit_tool_outputs", runStatus.required_action)
        if runStatus.required_action.submit_tool_outputs and runStatus.required_action.submit_tool_outputs.tool_calls:
            print("toolCalls present:")
            toolCalls = runStatus.required_action.submit_tool_outputs.tool_calls

            tool_outputs_result = []
            for toolcall in toolCalls:
                function_name = toolcall.function.name
                function_args = json.loads(toolcall.function.arguments)
                
                if function_name in available_functions:
                                        
                    function_to_call = available_functions[function_name]
                    print(function_to_call,function_to_call.__name__=="getCurrentWeather","================================================================")
                  
                    if function_to_call.__name__ == "getCurrentWeather":
                        
                        response = function_to_call(
                        location=function_args.get("location"),
                        unit=function_args.get("unit")
                        )
                        
                        
                        tool_outputs_result.append({
                                  "tool_call_id": toolcall.id,
                                  "output": response
                              })
                    
                    elif function_to_call.__name__ == "getNickname":
                        response = function_to_call(
                          location=function_args.get("location")
                          )
                        tool_outputs_result.append({
                          "tool_call_id": toolcall.id,
                          "output": response,
                              })
                    
                    elif function_to_call.__name__ == "get_flight_info":
                        response = function_to_call(
                          loc_origin=function_args.get("loc_origin"),
                          loc_destination=function_args.get("loc_destination")
                          )
                        tool_outputs_result.append({
                          "tool_call_id": toolcall.id,
                          "output": response,
                              })
            print(tool_outputs_result,">>>>>") 
            # Submit tool outputs and update the run
            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs_result
                )
      
    elif runStatus.status == "completed":
        # List the messages to get the response
        print("completed...........logic")
        messages: list[ThreadMessage] = client.beta.threads.messages.list(thread_id=thread.id)
        for message in reversed(messages.data):
            role_label = "User" if message.role == "user" else "Assistant"
            message_content = message.content[0].text.value
            print(f"{role_label}: {message_content}\n")
        break  # Exit the loop after processing the completed run

    elif run.status == "failed":
      print("Run failed.")
      break

    elif run.status in ["in_progress", "queued"]:
      print(f"Run is {run.status}. Waiting...")
      time.sleep(5)  # Wait for 5 seconds before checking again

    else:
      print(f"Unexpected status: {run.status}")
      break

requires_action .....
requires_action .....
Status:  requires_action


'submit_tool_outputs'

{'submit_tool_outputs': {'tool_calls': [{'id': 'call_pnAWRWMTttsNW2ayxrCKeNFn',
    'function': {'arguments': '{"location": "Tokyo"}', 'name': 'getNickname'},
    'type': 'function'},
   {'id': 'call_GNUBLSSxEJsjDoO6ZmMxcLe0',
    'function': {'arguments': '{"location": "Tokyo", "unit": "c"}',
     'name': 'getCurrentWeather'},
    'type': 'function'},
   {'id': 'call_58rfFvy4yA0K8TKOlh31gUWI',
    'function': {'arguments': '{"loc_origin": "LHE", "loc_destination": "KHI"}',
     'name': 'get_flight_info'},
    'type': 'function'}]},
 'type': 'submit_tool_outputs'}

toolCalls present:
<function getNickname at 0x000001A1622671A0> False ================================================================
<function getCurrentWeather at 0x000001A162267100> True ================================================================
<function get_flight_info at 0x000001A162267240> False ================================================================
[{'tool_call_id': 'call_pnAWRWMTttsNW2ayxrCKeNFn', 'output': 'tk'}, {'tool_call_id': 'call_GNUBLSSxEJsjDoO6ZmMxcLe0', 'output': '{"location": "Tokyo", "temperature": "10", "unit": "celsius"}'}, {'tool_call_id': 'call_58rfFvy4yA0K8TKOlh31gUWI', 'output': '{"loc_origin": "LHE", "loc_destination": "KHI", "datetime": "2023-12-08 19:03:32.255052", "airline": "AirBlue", "flight": "AB643"}'}] >>>>>
in_progress .....
Run is queued. Waiting...
completed .....
completed...........logic
User: What is the next flight from Paris to Tokyo?

Assistant: The next flight from Paris (CDG) to Tokyo (HND) is on December 8th, 2023 at 18:5